Generators
A generator is a function that can be paused and resumed while still maintaining state between these stops and starts.

Pausing and Resuming Functions?
Typically, when you call a function, you lose that function's local variables after it reaches the return statement.

Generators allow you to return a value
suspend execution of the function
then resume it later with all of the locals still intact!

Creating a Generator

*make a function that has the keyword 'yield' in it
*yield is like return in that gives the value immediately to the right of it
*however instead of stopping the functioin completely and discarding the locals
*it temporarily suspends the execution of the function so that it can be continued later

Generators and Iterators
From the docs:
When you call a generator function, it doesn’t return a single value; instead it returns a generator object that supports the iterator protocol.
So... a generator object is an iterator; it implements both __iter__ and __next__ (but an iterator is not always necessarily a generator)

Generators and Iterators Continued
So, when you call a generator function, you immediately get a generator object back, but the function body itself is not yet executed. The generator object returned behaves like an iterator:

it has a __next__ method
...so that means you can pass the generator object into the next function, similar to iterable objects returning iterators
using next controls the function's execution; it starts or resumes the function until yield is encountered
at which point a value is returned and execution is temporarily suspended.

In [2]:
#Generator function example

def f():
    print('print 1')
    yield 'return 1'
    print('print 2')
    yield 'return 2'
    print('print 3')
    yield 'return 3'

# when calling the generator function the body isn't executed
gen_obj = f()

# calling next starts/resumes function execution until yield is encountered
# note that 
next (gen_obj)
next (gen_obj)
next (gen_obj)

#can be looped over

for val in f():
    print(val)


print 1
print 2


'return 2'

No Class Needed!
Hm - this seems really similar to creating a class and implementing __iter__ and __next__. Aaaand, that's true:

generators are a simple way of getting an object back that supports the iterator protocol
no need to define a whole new class and define two methods on that class
just write a function
Let's write some code that allow us to loop over the letters in the alphabet without creating a string of the entire alphabet beforehand.

In [4]:
#example with a class

class Alphabet:
    START, STOP = 65, 91
    def __init__(self):
        self.i = Alphabet.START
        
    def __iter__(self):
        return self
    
    def __next__(self):
        ch = chr(self.i)
        self.i += 1
        if self.i > Alphabet.STOP:
            raise StopIteration
        return ch
    
for letter in Alphabet():
    print(letter)

A
B
C
D
E
F
G
H
I
J
K
L
M
N
O
P
Q
R
S
T
U
V
W
X
Y
Z


In [ ]:
#exmaple with a generator

def alphabet():
    START, STOP = 65, 91
    i = START
    while i < STOP:
        yield chr(i)
        i += 1
    # or use range, of course

for letter in alphabet():
    print(letter)

#infinite generator

def infinite_abc():
    START, STOP = 65,67
    i = START
    while True:
        if i > STOP:
            i = START
        yield chr(i)
        i += 1

In [ ]:
import sys
lc = [i ** 2 for i in range(10000)]
ge = (i ** 2 for i in range(10000))
sys.getsizeof(lc)
sys.getsizeof(ge)



In [ ]:
%%html
<style type="text/css">

.reveal table {
    font-size: 1em;
}

.reveal div.highlight {
    margin: 0; 
}

.reveal div.highlight>pre {
    margin: 0; 
    width: 100%;
    font-size: var(--jp-code-font-size);
}

.reveal div.jp-OutputArea-output>pre {
    margin: 0; 
    width: 90%;
    font-size: var(--jp-code-font-size);
    box-shadow: none;
}

</style>
